reference: https://hongl.tistory.com/290

In [1]:
class Grade:

    def __init__(self):
        self._value = 0

    def __get__(self, inst, inst_type):
        return self._value
    
    def __set__(self, inst, value):
        if not (0 <= value <= 100):
            raise ValueError(f"!")

        self._value = value

    

In [2]:
# 아래에 기술된 방식은 강사의 Pythonic 방식이고 
# 이것은 JAVA에서 Has-a relationship에서 착안해온 방식이다.
class yee:
    def __init__(self):
        self.a1 = Grade()
        self.a2 = Grade()
# 아래의 방식은 선언하기를 class variable로 선언해 놓고 인스턴스 변수로 쓰려고 해서 혼란스럽다.
# 그냥 나처럼 처음부터 init 안에서 선언하면 저런 문제도 겪지도 않는다.

        self._a3 = Grade()
        self.__a4 = Grade()


In [3]:
y1 = yee()
y2 = yee()
y1.a1 = 1
y1.a2 = 2

print(y1.a1, y1.a2)

y2.a1= 3
y2.a2 = 4
print(y1.a1, y1.a2, y2.a1, y2.a2)
print(f'y1 is y2 : {y1 is y2}')
print(f'y1.a1 is y2.a1 : {y1.a1 is y2.a1}')
print(f'y1.a1 is y1.a2 : {y1.a1 is y1.a2}')
print(f'y1.a1 is y2.a2 : {y1.a1 is y2.a2}')

1 2
1 2 3 4
y1 is y2 : False
y1.a1 is y2.a1 : False
y1.a1 is y1.a2 : False
y1.a1 is y2.a2 : False


In [19]:
y1.a3 = 101
y2.a3 = 202
y1.a4 = 303
y2.a4 = 404

print(y1.a3, y1.a4, y2.a3, y2.a4)
print(f'y1.a3 is y2.a3 : {y1.a3 is y2.a3}')
print(f'y1.a4 is y2.a4 : {y1.a4 is y2.a4}')
print(y1.__dict__)

101 303 202 404
y1.a3 is y2.a3 : False
y1.a4 is y2.a4 : False
{'a1': 1, 'a2': 2, '_a3': <__main__.Grade object at 0x0000025DACFDFF70>, '_yee__a4': <__main__.Grade object at 0x0000025DACFDFD30>, 'a3': 101, 'a4': 303}


그러나 문제가 발생했다. a1, a2, a3, a4를 보면 알겠지만 그냥 이름표를 새 변수로 옮겨 붙였을 뿐이고 Grade 객체의 상태값을 바꾼 것이 아니다. <br>
_a3, __a4는 변수의 이름이 바뀌었기 때문에 그대로 두고 새 변수만 만들어져 버렸다. <br>
그렇다고 별도의 사용자가 만든 getter, setter를 써야 한다. <br>
decorator인 @property 같은 것을 사용하면 kernel 충돌이 발생해서 정지된다. <br>
만일 JAVA의 방식대로 하고 싶다면 descriptor_supplementary2 파일을 확인해봐라.

Exam.__dict__['writing_grade'].__set__(exam1, 82)<br>
Exam.__dict__['writing_grade'].__set__(exam2, 11)<br>
지금 class 속을 보면 exam1, exam2 객체를 넣은 건 형식이고 사용되지 않는다. <br>
즉, 클래스에 하나만 생성되어 객체들에 공유된다.

In [5]:
class Exam:

    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()
    # 선언하기를 클래스 변수로 선언했으니 특별한 조치를 취하지 않으면 클래스 변수가 되어 모든 객체에 의해 공유되는 것이 당연하다.


In [6]:
exam1 = Exam()
exam1.writing_grade = 82  # Exam.__dict__['writing_grade'].__set__(exam1, 82)
exam1.science_grade = 99
print(f'exam1.writing_grade : {exam1.writing_grade}, exam1.science_grade : {exam1.science_grade}')

exam1.writing_grade : 82, exam1.science_grade : 99


In [7]:
exam2 = Exam()
exam2.writing_grade = 11
exam2.science_grade = 22
print(f'exam2.writing_grade : {exam2.writing_grade}, exam2.science_grade : {exam2.science_grade}')

exam2.writing_grade : 11, exam2.science_grade : 22


In [8]:
print(f'exam1.writing_grade : {exam1.writing_grade}, exam1.science_grade : {exam1.science_grade}')

# exam2를 변경했는데 exam1 값이 같이 변경된다. 이것은 얕은 복사의 문제다.
# 새로운 Grade instance가 생성되지 않았기 때문이다.
print('exam1 is exam2 ??', exam1 is exam2)
print('exam1.writing_grade is exam2.writing_grade ??', exam1.writing_grade is exam2.writing_grade)
print('exam1.science_grade_grade is exam2.science_grade ??', exam1.science_grade is exam2.science_grade)
print('exam1.writing_grade is exam2.science_grade ??', exam1.writing_grade is exam2.science_grade)
print('exam1.writing_grade is exam1.science_grade ??', exam1.writing_grade is exam1.science_grade)
# Shallow copy

exam1.writing_grade : 11, exam1.science_grade : 22
exam1 is exam2 ?? False
exam1.writing_grade is exam2.writing_grade ?? True
exam1.science_grade_grade is exam2.science_grade ?? True
exam1.writing_grade is exam2.science_grade ?? False
exam1.writing_grade is exam1.science_grade ?? False


문제는 writing_grade class_attribute로 한 Grade instance를 모든 Exam 인스턴스가 공유한다는 점이다.
프로그램이 실행되는 동안 Exam 클래스가 처음 정의될 때, 이 attribute에 대한 Grade instace가 단 한 번만 생성되고 Exam 인스턴스가 생성될 때마다 매번 Grade 인스턴스가 생성되지 않는다.

In [9]:
# property 제약 조건 추가

class SampleB:

    def __init__(self):
        self.x = 0
        self.__y = 0
        self._z = 0

    @property
    def y(self):
       
        return self.__y

    @y.setter
    def y(self, value):
        if value < 0:
            raise ValueError('0보다 큰 값을 넣어라!')
        self.__y = value

    @y.deleter
    def y(self):
      
        del self.__y

In [10]:
a1 = SampleB()
a2 = SampleB()

a1.x = 1
a2.x = 2

print(a1.x, a2.x)

a2.x = 6
print(a1.x, a2.x)

# 그냥 property decorator를 쓰면 이런 문제는 안 발생한다.

print(f'a1 is a2 : {a1 is a2}')
print(f'a1.x is a2.x : {a1.x is a2.x}')


1 2
1 6
a1 is a2 : False
a1.x is a2.x : False


Exam2 instance에 대해 값을 따로 추적하게 해야 하며 인스턴스별 상태를 딕셔너리에 저장하면 구현이 가능하다.
즉, Exam2 class의 attribute에 대해서 Grade2 인스턴스가 한 번 생성되지만 Exam2 인스턴스별로 값을 별도로 저장하는 것이다.

In [11]:
class Grade2:

    def __init__(self):
        self._value = {}

    def __get__(self, instance, isinstance_type):
        if instance is None:
            return self
        return self._value.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError(f"!")

        self._value[instance] = value

내 추측: 모든 인스턴스 변수는 클래스 변수를 딕셔너리로 추적하는 방식으로 확장한 것에 불과한 것이다. <br>
def __get__(self, instance, isinstance_type)에서 self는 Grade2의 객체이고 instance는 Exam2의 instance로 이번 Grade에선 instance를 따로 사용하지 않았기 때문에 class_variable로 사용되었다. <br>
왜냐하면 객체별로 구분할 수 없기 때문에 모든 instance 전체에서 공유하는 클래스 변수가 되어 버렸다. 클래스 변수인 Grade2 instance의 Grade2._value가 만들어진다. 그러나 각 Exam2의 instance를 key로 삼아서 <br>
클래스 변수 내부에서 객체별로 구분했기 때문에 instnace 변수로써 사용 가능한 것이다. <br>

In [12]:
class Exam2():
    
    writing_grade = Grade2()
    science_grade = Grade2()
    # class variable로 선언되었다. writing_grade, science_grade는 클래스 변수이므로 각 instance에서 공유될 뿐이고
    # Grade2 객체 안에 있는 딕셔너리에서 Exam2 인스턴스롤 key로 삼아서 구분되는 것이다.

    

In [13]:
exam3 = Exam2()


exam3.writing_grade = 88 # Exam2.__dict__['writing_grade'].__set__(exam3, 88)
exam3.science_grade = 70

print(exam3.writing_grade, exam3.science_grade)

exam4 = Exam2()

exam4.writing_grade = 11
exam4.science_grade = 22

print(exam4.writing_grade, exam4.science_grade)
print(exam3.writing_grade, exam3.science_grade)

88 70
11 22
88 70


In [14]:
print(f'exam3 is exam4 : {exam3 is exam4}')
print(f'exam3.science_grade is exam4.science_grade : {exam3.science_grade is exam4.science_grade}')

print(Exam2.writing_grade)
# 내가 선언한 객체는 2개임에도 불구하고 Exam2 class에는 writing_grade 객체가 1개만 있다. 즉, class 변수로 모든 객체에 공유되며 그 안에서 딕셔너리로 공유되고 있을 뿐이다.

exam3 is exam4 : False
exam3.science_grade is exam4.science_grade : False
